In [1]:
import os
import openai
import json
from Keys import openai_keys
import re

openai.organization = openai_keys['organization']
openai.api_key = openai_keys['api_key']

In [12]:
# Load the JSON data from the file
with open('Source/reddit-oppenheimer-26.json', 'r') as json_file:
    comments_by_post = json.load(json_file)

In [13]:
class Post:
    def __init__(self, title, analysis, raw):
        self.title = title
        self.analysis = analysis
        self.raw = raw

In [14]:
redditSentiment = list()
iter = 0
for title, comments in comments_by_post.items():
    prompts = [{"role": "system", "content":
        'Analyse the sentiment of 16 Reddit comments under a post - "' + str(
            title) + '", use the delimiter "\n\n==COMMENT==\n\n" to distinguish individual comments. Your goal is to write sentiment analysis for each comment towards the topic - "movie Oppenheimer", in a format RATING - KEYWORDS. Where RATING is a number from 0 to 10, with 0 representing drastically negative attitude to the topic, 10 for a drastically positive, or 5 for neutral or a comment unrelated to a topic. KEYWORDS are 2-3 words or phrases taken unchanged from a comment, which are most indicative of reasons for such attitude, separate them with a comma (,). Use a semicolon (;) as the delimiter between each comment analysis. Example: "8 - Exciting visuals, Nolan, direction; 4 - Confusing plot, slow pacing; 5 - long duration, too many characters;"'}]
    delimeter = "\n\n==COMMENT==\n\n"
    prompt = delimeter.join(comments)
    prompts.append({"role": "user", "content": prompt})
    chat = openai.ChatCompletion.create(model="gpt-3.5-turbo-16k", messages=prompts)
    response = chat.choices[0].message.content
    redditSentiment.append(Post(title, response.split(';'), prompt))
    iter += 1
    print(str(iter) + "/" + str(len(comments_by_post)))

1/19
2/19
3/19
4/19
5/19
6/19
7/19
8/19
9/19
10/19
11/19
12/19
13/19
14/19
15/19
16/19
17/19
18/19
19/19


In [32]:
rating = 0
keywords = dict()
for rPost in redditSentiment:
    tempRating = 0
    for com in rPost.analysis:
        if len(com) <= 3:
            continue
        numbers = re.findall(r'\d+', com)
        if len(numbers) > 0:
            tempRating += int(numbers[0])
        kwords = com.replace("\n", "").split(' - ')[1]
        kwords = kwords.split(',')
        kwords = [i.strip(" ") for i in kwords]
        for i in kwords:
            tmp = i.strip(" ").lower()
            if tmp in keywords:
                keywords[tmp] += 1
            else:
                keywords[tmp] = 1
    tempRating /= len(rPost.analysis)
    rating += tempRating
rating /= len(redditSentiment)

In [33]:
srted = list(reversed(sorted(keywords.items(), key=lambda item: item[1])))

In [38]:
print(redditSentiment[1].raw)

Margot Robbie vs mass destruction weapons.

==COMMENT==

Atomic bomb and blonde bombshell. I am become Doll, destroyer of young girls self image.

==COMMENT==

People have waaaaaay too much time on their hands

==COMMENT==

“I am become Barbie, destroyer of Ben Shapiro”

==COMMENT==

Osama Bin Laden and Bratz movies next year please.

==COMMENT==

Oppenheimer will stoke even more outrage once the muppets realize it doesn't actually glorify American militarism and nuclear weapons.

==COMMENT==

Guess which one has full frontal nudity.

==COMMENT==

Conservatives when a filmmaker that frequently discusses feminism in her films discusses feminism in her films: 🤯🤯

==COMMENT==

Ah yes, one of the most politcal controversy: plastic

==COMMENT==

Weirdly both of them.

==COMMENT==

Shouldn't have touched east Vietnam/ west Philippines sea.

==COMMENT==

Can we stop pretending that this is "political outrage"? The right wing grifters probably made their review videos after the first trailer d

In [39]:
print(redditSentiment[1].analysis)

['7 - destroyer of young girls self image, Doll', ' 5 - too much time', ' 6 - destroyer of Ben Shapiro', ' 5 - Osama Bin Laden, Bratz movies', " 9 - doesn't actually glorify American militarism and nuclear weapons", ' 7 - full frontal nudity', ' 2 - conservatives, feminism in films', ' 5 - plastic controversy', ' 5 - unrelated', ' 6 - political controversy, east Vietnam/ west Philippines sea', ' 6 - right wing grifters, review videos', ' 7 - feminism, social statements, patriarchy', ' 4 - anti-barbie people, outrage merchants', " 9 - Schindler's list, factory", ' 8 - political expectations from a plastic doll', '']


In [40]:
redditSentiment[1].title

"Oppenheimer (2023) and Barbie (2023) open the same day. One is about the invention of the atomic bomb. The other is about a plastic doll. Guess which one stoked the most political outrage. Go on, take a wild fuckin' guess"

In [8]:
from IPython.display import Markdown, display

prompts = [{"role": "system", "content":
    'You are a software development specialist'}]

In [13]:
prompt = input()
prompts.append({"role": "user", "content": prompt})
chat = openai.ChatCompletion.create(model="gpt-4", messages=prompts)
response = (chat.choices[0].message.content)
display(Markdown(response))
prompts.append({"role": "assistant", "content": response})

To get the subtitles (also known as closed captions) for a YouTube video, `captions().list` method provided by Youtube Data API v3 can be used. 

For more details, check the official YouTube API documentation: https://developers.google.com/youtube/v3/docs/captions

However, please note that not all videos have available closed captions, and some may have multiple sets of captions in different languages. You can specify the language of the subtitle that you want.

But it's important to note that downloading YouTube captions requires proper authorization with OAuth 2.0 using the 'https://www.googleapis.com/auth/youtube.force-ssl' scope.

Here is a simple demonstration code. But remember, for proper OAuth 2.0 setup and usage, please refer to the official Google OAuth 2.0 guide:

```python
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.credentials import Credentials
import os
import io

# Load credentials from the 'token.json' file
creds = Credentials.from_authorized_user_file('token.json')

youtube = build('youtube', 'v3', credentials=creds)

def get_captions(youtube, video_id):
    try:
        # Retrieve caption tracks list
        captions = youtube.captions().list(
            part="snippet",
            videoId=video_id
        ).execute()

        # Select the first caption track
        caption = captions['items'][0]

        # Download the caption track in the language you want
        subtitle = youtube.captions().download(
            id=caption['id'],
            tfmt='srt' # VTT or SRT subtitle formats.
        ).execute()

        # Save the subtitle to a .srt (or .vtt) file
        with open("subtitle.srt", "w", encoding="utf-8") as f:
            f.write(subtitle)

        print("Subtitle has been written to subtitle.srt")

    except HttpError as e:
        print(f"A HTTP error {e.resp.status} occurred:\n{e.content}")

get_captions(youtube, 'VIDEO_ID') # place your video id here
```

In the code snippet above:

1. We are loading the user credentials from 'token.json'. You should replace this with the path to your OAuth 2.0 credentials file. You can generate this file using Google's OAuth 2.0 Playground and save it to your script's directory.

2. You need to replace "VIDEO_ID" with your actual video ID.

Please keep in mind that reading the captions of a Youtube video with the API costs 200 quota units, and writing a simple list operation costs 50. Always be mindful of Google's daily quota limits when using their API services.

In [11]:
display(Markdown(prompts[2]['content']))

To get the IDs of all Youtube videos from a specific channel, you can use the YouTube API with the Google API client for Python. Here's a basic script to get you started:

```python
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

api_key = 'YOUR_API_KEY' # replace with your own API key
youtube = build('youtube', 'v3', developerKey=api_key)

def get_channel_videos(channel_id):
    res = youtube.channels().list(id=channel_id, part='contentDetails').execute()
    playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    videos = []
    next_page_token = None

    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id, 
                                           part='snippet', 
                                           maxResults=50,
                                           pageToken=next_page_token).execute()
        videos += res['items']
        next_page_token = res.get('nextPageToken')

        if next_page_token is None:
            break

    return videos

videos = get_channel_videos('CHANNEL_ID') # replace with the channel id 
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))

print("\n".join(video_ids))
```

This script will retrieve all the videos from the specified channel's Uploads playlist, then extract the video ID from each video in that playlist.

Important points to note:
- Replace `'YOUR_API_KEY'` with your actual API key that you obtained from the Google Developer Console.
- Replace `'CHANNEL_ID'` with the ID of the YouTube Channel from which you want to fetch all video IDs.
- API quotas mean you can only make a certain number of requests per day - if you are exceeding your quota, you may need to wait, or request additional quota.
- Depending on the number of videos, fetching all video IDs might take some time. If request times out or you encounter a similar problem, consider fetching in batches or using pagination.
